# Setup

In [ ]:
import numpy as np
import open3d as o3d
from sklearn import cluster

from PointCloudDataFrame import PointCloudDataFrame

In [ ]:
pcd_raw = o3d.io.read_point_cloud('point_cloud.pts')

In [ ]:
pcd = pcd_raw.voxel_down_sample(voxel_size=0.001)  # 2mm
df = PointCloudDataFrame.from_pcd(pcd)

# o3d.visualization.draw_geometries([pcd])

In [ ]:
df_filter1 = df[(df['s'] < 0.075) & (df['v'] > 0.2)]
df_filter1.write_pts('output/filter1.pts')
pcd_filter1 = df_filter1.to_pcd()

# o3d.visualization.draw_geometries([pcd_filter1])

In [ ]:
df_filter2 = df[(df['s'] < 0.075) & (df['v'] > 0.2) & (df['z'] > 0.1)]
df_filter2.write_pts('output/filter2.pts')
pcd_filter2 = df_filter2.to_pcd()

# o3d.visualization.draw_geometries([pcd_filter2])

In [ ]:
_, ind = pcd_filter2.remove_radius_outlier(nb_points=56, radius=0.005)
pcd_inliers = pcd_filter2.select_by_index(ind)
pcd_inliers.paint_uniform_color([0.8, 0.8, 0.8])
df_inliers = PointCloudDataFrame.from_pcd(pcd_inliers)

# o3d.visualization.draw_geometries([pcd_inliers])

# K-Means Clustering

In [ ]:
model_kmeans = cluster.KMeans(n_clusters=128, n_init='auto')
model_kmeans.fit(pcd_inliers.points)

dictionary = {str(i): list(k) for (i, k) in enumerate(model_kmeans.cluster_centers_)}
json_object = json.dumps(dictionary)
with open('output/clusters.json', 'w') as f:
    f.write(json_object)

In [ ]:
clusters_kmeans = o3d.geometry.PointCloud()
clusters_kmeans.points = o3d.utility.Vector3dVector(model_kmeans.cluster_centers_)
o3d.io.write_point_cloud('output/clusters_kmeans.xyz', clusters_kmeans, write_ascii=True)

In [ ]:
# o3d.visualization.draw_geometries([pcd_inliers, clusters_kmeans])

# DBSCAN Clustering

In [ ]:
model_dbscan = cluster.DBSCAN(eps=0.002, min_samples=10)
model_dbscan.fit(pcd_inliers.points)

In [ ]:
labels = model_dbscan.labels_ 
num_labels = len(set(labels).difference({-1}))
print(num_labels)

model_dbscan_centers = []
for i in range(num_labels):
    model_dbscan_centers.append(
        np.mean(df_inliers[labels==i][['x', 'y', 'z']], axis=0)
    )

clusters_dbscan = o3d.geometry.PointCloud()
clusters_dbscan.points = o3d.utility.Vector3dVector(model_dbscan_centers)
o3d.io.write_point_cloud('output/clusters_dbscan.xyz', clusters_dbscan, write_ascii=True)

In [ ]:
o3d.visualization.draw_geometries([pcd_inliers, clusters_dbscan])